<center>Python课程设计</center>
<br>
<br>
<center>选题：蔡徐坤粉丝数及转发数据真假状况分析</center>
<br>
<br>
<center>小组成员：徐佳清、章华鼎、张湖川</center>

# 1.分析目标
1. 蔡徐坤的微博转发是否存在假流量？
2. 真假流量所占的比例各有多少？
3. 假流量粉丝是如何生产出来的？
4. 真流量粉的粉丝画像

# 2.分析内容
    读取转发数据的全部信息，进行数据清洗，然后对转发数据进行统计分析，最后实现数据可视化分析。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pyecharts import Bar
%matplotlib inline

plt.style.use('ggplot')
mpl.rcParams['font.sans-serif'] = ['SimHei']  #解决seaborn中文字体显示问题
plt.rc('figure', figsize=(10, 10))  #把plt默认的图片size调大一点
plt.rcParams["figure.dpi"] =mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

In [2]:
#读取蔡徐坤100万+转发的微博《再见，“任性的”千千…》的10万条转发数据CaiXuKun.csv
data = pd.read_csv('D:\Desktop\CaiXuKun\caixukun.csv', encoding='UTF-8')    

In [3]:
#检查数据以确保数据运行正常
data.sample(5)

,attitudes_count,comments_count,reposts_count,mid,raw_text,source,user.description,user.follow_count,user.followers_count,user.gender,user.id,user.mbrank,user.mbtype,user.profile_url,user.profile_image_url,user.screen_name,user.statuses_count,user.urank,user.verified,user.verified_reason
11326,0,0,0,4348295655167484,With the wonder of your love， the sun above al...,Android,NaN,0,1,m,7011853676,0,0,https://m.weibo.cn/u/7011853676?uid=7011853676,https://tvax1.sinaimg.cn/crop.0.0.640.640.180/...,爱上徐坤TL2113,30,3,False,NaN
45065,0,0,0,4348426768606383,[羞嗒嗒]//@是你的茵茵吖:#东方风云榜让世界看见蔡徐坤#‖#蔡徐坤的未完成# 要快乐成...,Android,NaN,0,1,m,7003550626,0,0,https://m.weibo.cn/u/7003550626?uid=7003550626,https://tvax1.sinaimg.cn/crop.0.0.100.100.180/...,用户7003550626,50,4,False,NaN
25403,0,0,0,4348389859177814,星光不问赶路人，时光不负蔡徐坤。因为不再是单枪匹马，所以不要再害怕绽放光芒@蔡徐坤,Android,NaN,0,1,m,6969183796,0,0,https://m.weibo.cn/u/6969183796?uid=6969183796,https://tvax2.sinaimg.cn/crop.0.0.240.240.180/...,我们都来自地球fL9342,5,2,False,NaN
27396,0,0,0,4348387442516268,"I'm not myself, lately,I'm foolish //@蔡三岁cai三岁...",荣耀手机 勇敢做自己,NaN,0,1,m,6975435399,0,0,https://m.weibo.cn/u/6975435399?uid=6975435399,https://tvax4.sinaimg.cn/crop.0.0.100.100.180/...,蔡徐坤LFi822,92,4,False,NaN
86979,0,0,0,4348631999922456,"Life, thin and light-off time and time again",Android,NaN,0,1,m,7011853645,0,0,https://m.weibo.cn/u/7011853645?uid=7011853645,https://tvax3.sinaimg.cn/crop.0.0.640.640.180/...,大胖白菜6lI386,31,3,False,NaN


In [4]:
#查看数据基本情况
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102313 entries, 0 to 102312
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   attitudes_count         102313 non-null  int64 
 1   comments_count          102313 non-null  int64 
 2   reposts_count           102313 non-null  int64 
 3   mid                     102313 non-null  int64 
 4   raw_text                102313 non-null  object
 5   source                  102188 non-null  object
 6   user.description        4569 non-null    object
 7   user.follow_count       102313 non-null  int64 
 8   user.followers_count    102313 non-null  int64 
 9   user.gender             102313 non-null  object
 10  user.id                 102313 non-null  int64 
 11  user.mbrank             102313 non-null  int64 
 12  user.mbtype             102313 non-null  int64 
 13  user.profile_url        102313 non-null  object
 14  user.profile_image_url  102313 non-n

由于数据入库的时候没有进行清洗，所以数据多出了很多没用的字段，需要先清洗掉

In [5]:
#返回列
data.columns

Index(['attitudes_count', 'comments_count', 'reposts_count', 'mid', 'raw_text',
       'source', 'user.description', 'user.follow_count',
       'user.followers_count', 'user.gender', 'user.id', 'user.mbrank',
       'user.mbtype', 'user.profile_url', 'user.profile_image_url',
       'user.screen_name', 'user.statuses_count', 'user.urank',
       'user.verified', 'user.verified_reason'],
      dtype='object')

选取需要的字段：

In [6]:
in_columns = ['attitudes_count', 'comments_count', 'reposts_count', 'mid', 'raw_text', 
          'source', 'user.description', 'user.follow_count', 'user.followers_count', 
          'user.gender', 'user.id', 'user.mbrank', 'user.mbtype', 'user.profile_url', 
          'user.profile_image_url', 'user.screen_name', 'user.statuses_count', 
          'user.urank', 'user.verified', 'user.verified_reason']
data = data[in_columns]

In [7]:
#再次查看数据基本情况
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102313 entries, 0 to 102312
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   attitudes_count         102313 non-null  int64 
 1   comments_count          102313 non-null  int64 
 2   reposts_count           102313 non-null  int64 
 3   mid                     102313 non-null  int64 
 4   raw_text                102313 non-null  object
 5   source                  102188 non-null  object
 6   user.description        4569 non-null    object
 7   user.follow_count       102313 non-null  int64 
 8   user.followers_count    102313 non-null  int64 
 9   user.gender             102313 non-null  object
 10  user.id                 102313 non-null  int64 
 11  user.mbrank             102313 non-null  int64 
 12  user.mbtype             102313 non-null  int64 
 13  user.profile_url        102313 non-null  object
 14  user.profile_image_url  102313 non-n

#没啥好洗的，干净的很😄

# 4.数据分析

### 4.1. 蔡徐坤的微博转发是否存在假流量？

In [8]:
# 先来看看蔡徐坤的粉丝性别比例
fans_num = data['user.gender'].value_counts()
fans_num

m    93618
f     8695
Name: user.gender, dtype: int64

In [9]:
#利用Bar显示性别比例可视化
bar = Bar("蔡徐坤粉丝性别比例初探", width = 600,height=500)
bar.add("(总数据102313条)", ['男', '女'], fans_num.values, is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [10]:
#男女比例
np.round(fans_num/fans_num.sum()*100, 2)

m    91.5
f     8.5
Name: user.gender, dtype: float64

In [11]:
#随机显示5组性别为m的用户的信息
data[data['user.gender']=='m'].sample(5)

,attitudes_count,comments_count,reposts_count,mid,raw_text,source,user.description,user.follow_count,user.followers_count,user.gender,user.id,user.mbrank,user.mbtype,user.profile_url,user.profile_image_url,user.screen_name,user.statuses_count,user.urank,user.verified,user.verified_reason
50496,0,0,0,4348430392776323,有喜欢的人真好，晴天适合见面，雨天适合想念@蔡徐坤//@IKUN夏天:#东方风云榜让世界看见...,Android,NaN,0,1,m,7020503255,0,0,https://m.weibo.cn/u/7020503255?uid=7020503255,https://tvax4.sinaimg.cn/crop.0.21.551.551.180...,大奎好酷Wc5018,7,2,False,NaN
16246,0,0,0,4348317938403448,Bad news travels fast.,Android,NaN,0,1,m,6927604090,0,0,https://m.weibo.cn/u/6927604090?uid=6927604090,https://tvax3.sinaimg.cn/default/images/defaul...,用户6927604090,87,4,False,NaN
74865,0,0,0,4348595244731034,Solo.,前后2000万 OPPO R11s,NaN,0,1,m,7012394103,0,0,https://m.weibo.cn/u/7012394103?uid=7012394103,https://tvax4.sinaimg.cn/crop.0.0.640.640.180/...,爱坤是我RYk122,71,2,False,NaN
87034,0,0,0,4348651004434905,Weight weight weight//@是积极努力的心心唷:#东方风云榜让世界看见蔡徐...,Android,NaN,0,1,m,6941059879,0,0,https://m.weibo.cn/u/6941059879?uid=6941059879,https://tvax3.sinaimg.cn/crop.0.0.960.960.180/...,温柔坤哥855631,114,4,False,NaN
49711,0,0,0,4348405776452690,So I can't forget you //@小葵花迷糊:带着我们的爱好好长大吧[米奇比...,三星Galaxy NOTE III,NaN,0,1,m,6941601941,0,0,https://m.weibo.cn/u/6941601941?uid=6941601941,https://tvax4.sinaimg.cn/crop.0.0.1018.1018.18...,蜜桃葵zpQ181,12,2,False,NaN


### 4.2. 真假流量所占的比例各有多少？

In [12]:
#筛选出用户描述‘wu’
data=data.fillna({'user.description':'wu'})

In [13]:
data.sample(5)

,attitudes_count,comments_count,reposts_count,mid,raw_text,source,user.description,user.follow_count,user.followers_count,user.gender,user.id,user.mbrank,user.mbtype,user.profile_url,user.profile_image_url,user.screen_name,user.statuses_count,user.urank,user.verified,user.verified_reason
51472,0,0,0,4348415574167973,Were spent in ecstasy//@南南akunkun:千千,Android,wu,0,1,m,7016592631,0,0,https://m.weibo.cn/u/7016592631?uid=7016592631,https://tvax4.sinaimg.cn/crop.0.0.640.640.180/...,奶香坤mvY983,9,3,False,NaN
48198,0,0,0,4348423929020037,Hide not your light under a bushel.,Android,wu,0,2,m,6967589421,0,0,https://m.weibo.cn/u/6967589421?uid=6967589421,https://tvax4.sinaimg.cn/crop.0.0.200.200.180/...,世态苍凉别太善良哦iLMe272,218,4,False,NaN
21282,0,0,0,4348380597995203,Frivolous tireless//@石头打瞌睡:#东方风云榜让世界看见蔡徐坤# | ...,Android,wu,0,1,m,7012424097,0,0,https://m.weibo.cn/u/7012424097?uid=7012424097,https://tvax1.sinaimg.cn/crop.0.0.640.640.180/...,期待坤哥NU6307,68,3,False,NaN
100663,0,0,0,4348690229520304,Auxiliary extreme aestheticism bait to capture...,Android,wu,0,1,m,6960313265,0,0,https://m.weibo.cn/u/6960313265?uid=6960313265,https://tvax2.sinaimg.cn/crop.0.0.1080.1080.18...,小葵的花2Tw336,77,4,False,NaN
48626,0,0,0,4348405335563611,Sharp tools make good work.//@Ann_梅:我来了[酸]//@A...,Android,wu,0,1,m,7011899353,0,0,https://m.weibo.cn/u/7011899353?uid=7011899353,https://tvax4.sinaimg.cn/crop.0.0.640.640.180/...,生命粉tQH309,52,4,False,NaN


In [14]:
#通过一番探索分析，我们把转发数据中转发者的关注或者粉丝数少于等于5、筛选出用户描述为‘wu’，
#发表评论、转发之后被点赞数评论数再转发数都为0、微博会员等级为0级的数据，以及转发者的关注
#这部分数据，便是我们所说的假流量。

data_fake = data[((data['user.follow_count']<=5)|(data['user.followers_count']<=5))&
                 (data['user.description']=='wu')&
                 (data['comments_count']==0)&
                (data['attitudes_count']==0)&
                (data['reposts_count']==0)&
                (data['user.mbrank']==0)]
data_fake.sample(5)

,attitudes_count,comments_count,reposts_count,mid,raw_text,source,user.description,user.follow_count,user.followers_count,user.gender,user.id,user.mbrank,user.mbtype,user.profile_url,user.profile_image_url,user.screen_name,user.statuses_count,user.urank,user.verified,user.verified_reason
3622,0,0,0,4348258514121849,[并不简单]//@蔡徐坤葵葵坤坤奎奎都是我的我的:kun&amp;ikun#东方风云榜让世界...,Android,wu,0,1,m,7012421272,0,0,https://m.weibo.cn/u/7012421272?uid=7012421272,https://tvax2.sinaimg.cn/crop.0.0.640.640.180/...,坤哥最棒1UM945,18,4,False,NaN
70837,0,0,0,4348571463516288,"Da Satana o da Dio, che importa? Angelo o Sirena,",vivo X20全面屏手机,wu,0,1,m,7011819429,0,0,https://m.weibo.cn/u/7011819429?uid=7011819429,https://tvax1.sinaimg.cn/crop.0.0.640.640.180/...,享受舞台yqy910,69,3,False,NaN
54586,0,0,0,4348446691755632,so I can't forget you as much as I should //@听...,Android,wu,0,1,m,7016494976,0,0,https://m.weibo.cn/u/7016494976?uid=7016494976,https://tvax3.sinaimg.cn/crop.160.0.640.640.18...,坤小迷妹UGD712,13,4,False,NaN
62405,0,0,0,4348401846309654,I believe I am//@霸气姐姐e3://@AK47-KAKAKA:#东方风云榜让...,Android,wu,0,1,m,7011671940,0,0,https://m.weibo.cn/u/7011671940?uid=7011671940,https://tvax2.sinaimg.cn/crop.0.0.100.100.180/...,小煤球lgm009,45,4,False,NaN
81941,0,0,0,4348629546306897,La mia noia combatte con i lenti crepuscoli.,小米手机,wu,0,1,m,7011819040,0,0,https://m.weibo.cn/u/7011819040?uid=7011819040,https://tvax2.sinaimg.cn/crop.0.0.640.640.180/...,坤妹有矿8HU342,76,2,False,NaN


In [15]:
#我们可以看到此时假粉转发量已经到95326了！
data_fake.shape

(95326, 20)

In [16]:
#然后粉丝数大于等于5但昵称包含“用户”的这样的数据抽取出来。基本上也可以断定是假粉丝
data_fake2_index = data[(data['user.follow_count']>5)&
                        (data['user.followers_count']>5)&
                        (data['user.screen_name'].str.contains('用户'))].index

In [17]:
# 最后把假的流量粉丝转发组合起来
data_fake = pd.concat([data_fake, data.iloc[data_fake2_index]])

In [18]:
#此时假粉转发量已经有95397了
data_fake.shape

(95397, 20)

In [19]:
# 取出真粉的转发
data_true = data.drop(data_fake.index)

In [20]:
#这时真粉转发量有6916
data_true.shape

(6916, 20)

In [21]:
print('真粉丝转发数占总转发数的{}%'.format(np.round(data_true.shape[0]/data.shape[0]*100, 2)))
print('假粉丝转发数占总转发数的{}%'.format(np.round(data_fake.shape[0]/data.shape[0]*100, 2)))

真粉丝转发数占总转发数的6.76%
假粉丝转发数占总转发数的93.24%


In [22]:
#现在我们利用bar把蔡徐坤真假流量的转发量可视化！
bar = Bar("蔡徐坤真假流量的转发量", width = 600,height=500)
bar.add("(总数据102313条)", ['总转发量', '假粉丝转发量', '真粉丝转发量'], 
        [data.shape[0], data_fake.shape[0], data_true.shape[0]], is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

从上图我们可以看到，在102313条转发数据中，有95397条是由假粉丝转发的，占了总转发的93.24%，只有6916条是由真粉丝转发的，占6.76%。

   那么6919条真粉丝转发的数据中，除去重复转发刷榜的数量，里面一共有多少个真粉丝在转发呢？我们把这部分数据按照粉丝微博ID进行去重。

In [23]:
#真实转发的真粉丝！
real_fans_num = data_true.drop_duplicates(subset='user.id').shape[0]
real_fans_num

3926

In [24]:
#利用bar显示蔡徐坤真假流量的转发量与真实转发粉丝量
bar = Bar("蔡徐坤真假流量的转发量与真实转发粉丝量(总数据102313条)", width = 600,height=500)
bar.add('', ['总转发量', '假粉丝转发量', '真粉丝转发量', '真实转发粉丝量'], 
        [data.shape[0], data_fake.shape[0], data_true.shape[0], real_fans_num], is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True, xaxis_rotate=20)
bar

In [25]:
print('真实转发粉丝量占总转发数的{}%'.format(np.round(real_fans_num/data.shape[0]*100, 2)))

真实转发粉丝量占总转发数的3.84%


### 3. 假流量粉丝是如何生产出来的？【对data_fake进行讨论】

In [26]:
#假流量粉丝性别比例
data_fake_gender = data_fake.drop_duplicates(subset='user.id')['user.gender'].value_counts()
data_fake_gender

m    38969
f     1869
Name: user.gender, dtype: int64

In [27]:
#利用Bar实现假粉丝性别可视化
bar = Bar("蔡徐坤假粉丝性别比例", width = 600,height=500)
bar.add("(假粉丝总数为40838)", ['男', '女'], data_fake_gender.values, is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [28]:
38969/40838

0.954233801851217

从上图可见在95397条假转发中，有40838个假粉丝。其中男性的比例高达95.42%！

In [29]:
#再看一下假粉丝转发微博里都是些啥玩意！
data_fake['raw_text'].value_counts()

转发微博                                                                    429
I am only waiting for love to give myself up at last into his hands.    375
想你//@蔡徐坤的南岸末阴大小姐:#东方风云榜让世界看见蔡徐坤# /#蔡徐坤的未完成#祝千千在新家能快快乐乐 健健康康的@蔡徐坤        289
我心悦你//@蔡徐坤的南岸末阴大小姐:#东方风云榜让世界看见蔡徐坤# /#蔡徐坤的未完成#祝千千在新家能快快乐乐 健健康康的@蔡徐坤      288
爱你//@蔡徐坤的南岸末阴大小姐:#东方风云榜让世界看见蔡徐坤# /#蔡徐坤的未完成#祝千千在新家能快快乐乐 健健康康的@蔡徐坤        278
                                                                       ... 
你是我藏在枕中的春秋大梦                                                              1
蔡徐坤 cxk “下雨天了怎么办…我好想你 ”@蔡徐坤//@抡博小困:再见千千                                   1
青年朋友们，要敢于有梦，勇于追梦，勤于圆梦。//@Abcandghghgh:#东方风云榜让世界看见蔡徐坤# 菜菜我想你了              1
每一个能看见你的都日子，都是我的幸运日                                                       1
Open to the lonely soul of sickle harvesting                              1
Name: raw_text, Length: 12679, dtype: int64

In [30]:
#假粉丝使用的前10设备
fake_source = data_fake['source'].value_counts()[:10]
fake_source

Android              65495
Flyme                 2823
三星Galaxy NOTE III     2455
三星android智能手机         2354
红米Redmi               2197
vivo X20全面屏手机         1458
小米手机                  1254
荣耀手机 勇敢做自己            1225
前后2000万 OPPO R11      1219
HUAWEI P10            1166
Name: source, dtype: int64

In [31]:
#利用Bar实现假粉丝使用设备比例可视化
bar = Bar("蔡徐坤假粉丝Top10转发设备", width = 600,height=600)
bar.add("", fake_source.index, fake_source.values, is_stack=True, 
       xaxis_label_textsize=11, yaxis_label_textsize=14, is_label_show=True, xaxis_rotate=30)
bar

在假粉丝使用的Top10转发设备中，Android高居榜首，这也再次证明了这些粉丝是假粉丝。

另外还有一些有趣的发现（请接着看下面代码）：假粉丝的平均关注是3.44，平均粉丝数是1.04，没有简介，昵称基本上都是“中文+英文和数字”这个格式，很多假粉丝的昵称都带有“坤”、“蔡”、“葵”、“kun”等字，头像都是蔡徐坤（说明很多都是定制粉啊）。

下面假粉丝的一些信息分析

In [32]:
#假粉丝的平均关注数
data_fake['user.follow_count'].mean()

3.4412612555950397

In [33]:
#假粉丝的平均粉丝数
data_fake['user.followers_count'].mean()

1.04576663836389

In [34]:
#拿出5个假粉丝
data_fake_sample = data_fake.sample(5)

In [35]:
#随机5个假粉丝的用户名
data_fake_sample['user.screen_name']

53248      坤不敢当7yr545
50983    用户7020341526
43848      没读研_大蒜6fd9
12695      奶坤上线0zX863
98152       坤坤8beL802
Name: user.screen_name, dtype: object

In [36]:
#看一下假粉丝昵称中带有“坤”、“蔡”、“葵”、“kun”的总和
data_fake['user.screen_name'].str.contains('蔡|坤|葵|kun').sum()

41766

In [37]:
#假粉丝中平均在线状态
data_fake['user.statuses_count'].mean()

72.4942503433022

### 4. 真流量粉的粉丝画像

先来看看真粉丝的性别比例。可见3926个真粉丝中，女生（f）占了绝大部分，这才是符合逻辑的粉丝比例嘛。

In [38]:
data_true_gender = data_true.drop_duplicates(subset='user.id')['user.gender'].value_counts()
data_true_gender

f    3287
m     639
Name: user.gender, dtype: int64

In [39]:
#利用Bar实现真粉丝性别比例：
bar = Bar("蔡徐坤真粉丝性别比例", width = 600,height=500)
bar.add("(真粉丝总数为3926)", ['女', '男'], data_true_gender.values, is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [40]:
#转发微博评论
data_true['raw_text'].value_counts()

转发微博                                                                                                                   1045
@蔡徐坤 我永远支持你！我们一起拿下 #明星势力榜# 第一名！                                                                                         622
#东方风云榜让世界看见蔡徐坤#                                                                                                          73
@蔡徐坤  我在#明星ALL榜[超话]#上为你加油啦，你是我今生唯一的执著哦。#蔡徐坤[超话]# 棒棒哒！快来为TA应援吧                                                            50
//@蔡徐坤工作室:#蔡徐坤[超话]#[给你小心心]#蔡徐坤的未完成#之宠物医院终于步入尾声，在短暂的相处时间里，因为工作忙碌无法养育小动物的@蔡徐坤 也获得了片刻的慰藉，感谢千千给我们带来了一段难忘的回忆[心]#蔡徐坤 ONE#      42
                                                                                                                       ... 
#东方风云榜让世界看见蔡徐坤# 宝贝你的金海我们承包了@蔡徐坤                                                                                           1
#东方风云榜让世界看见蔡徐坤# [微笑]#蔡徐坤的未完成# 期待蔡徐坤的舞台[微笑]                                                                                1
阔耐#东方风云榜

在这些粉丝转发所携带的评论中可以看到，很多都是支持蔡徐坤拿下#明星势力榜#或者#东方风云榜#第一名而转发的。

In [41]:
true_source = data_true['source'].value_counts()[:10]

In [42]:
#利用Bar实现真粉丝使用设备比例：
bar = Bar("蔡徐坤真粉丝Top10转发设备", width = 600,height=600)
bar.add("", true_source.index, true_source.values, is_stack=True, 
       xaxis_label_textsize=11, yaxis_label_textsize=14, is_label_show=True, xaxis_rotate=30)
bar

真粉丝所使用的转发设备，各种设备分布都比较均匀，最受欢迎的是iPhone客户端。

下面是一些真粉丝中的信息，真粉丝的平均关注量为222，平均粉丝数是179，同样，很多粉丝名字中喜欢带有“坤”、“蔡”、“葵”、“kun”等字。

In [43]:
#真粉丝平均关注的人
data_true['user.follow_count'].mean()

222.0597165991903

In [44]:
#平均粉丝数
data_true['user.followers_count'].mean()

178.9480913823019

In [45]:
#随机取出5个真粉丝的名字
data_true.sample(5)['user.screen_name']

26727    ikun婷确认过眼神是坤的人
16889           蔡徐坤的耳朵疼
68884              坤困lu
48750              心心恰蔡
49142            最爱坤的橙子
Name: user.screen_name, dtype: object

In [46]:
#真粉丝名字中的一些特点
data_true['user.screen_name'].str.contains('蔡|坤|葵|kun').sum()

3153

我们把真粉丝的简介做成了词云图。

In [47]:
# 绘制蔡徐坤真粉丝的简介词云图
import jieba
from collections import Counter
from pyecharts import WordCloud

jieba.add_word('蔡徐坤')

swords = [x.strip() for x in open ('D:\Desktop\CaiXuKun\stopwords.txt', encoding='UTF-8')]

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Qing\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\Qing\AppData\Local\Temp\jieba.cache
Loading model cost 0.613 seconds.
DEBUG:jieba:Loading model cost 0.613 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [48]:
def plot_word_cloud(data, swords, columns):
    text = ''.join(data[columns])
    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    wordcloud = WordCloud(width=1300, height=620)
    wordcloud.add("", wc_data['word'], wc_data['counts'], word_size_range=[20, 100])
    return wordcloud

In [49]:
plot_word_cloud(data=data_true, swords=swords, columns='user.description')

可见，真粉丝们的简介都喜欢带上蔡徐坤的名字，很喜欢蔡徐坤，想要一直陪着他走下去。看着里面的“少年”、“努力”、“自由”“追梦”等字眼，发现这就是我们以前的青春呀。

另外还把转发所带的评论做成了词云图。

In [50]:
plot_word_cloud(data=data_true, swords=swords, columns='raw_text')

可见，粉丝们都很在乎“风云榜”这件事，并且要帮助蔡徐坤拿第一名。里面有早安打卡的，有超级话题的，还有很多“开心”、“比心”、“温暖”等字眼。说明绝大部分真粉还是很温暖的。

# <center>总结</center>                            

    数据显示，蔡徐坤的100万+的微博转发中，确实存在绝大部分的假流量。这些假流量估计有两方面的来源：一是自身经纪公司购买的，二是忠实的粉丝自费购买的。所以在这里提醒一下大家，追星需谨慎呀！🙂